In [46]:
import sys
sys.path.append('../_amigocloud')
from amigocloud import AmigoCloud

import pandas as pd
from datetime import datetime, timedelta
import collections

from docxtpl import DocxTemplate
from docx.shared import Mm
import docxtpl
import requests

In [47]:
# ruta de carpera donde estan las llaves
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]

# leer token
acceso = open(ruta + '/_keys/api_amigocloud.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

In [61]:
fecha_insp = '2025-02-25'
query = {'query': 'select id, canhero, propiedad, lote, variedad from dataset_365887 where fecha_registro::date=\'{fecha}\''.format(fecha=fecha_insp)}
select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/35567/sql', query)
data = select['data']
tabla = pd.DataFrame(data)
tabla

,id,canhero,propiedad,lote,variedad
0,151,4171 / FRITZER CUELLAR SAMIR EDUARDO,273 / HACIENDA LOS NIETOS--FRITZER,L7,UCG 90-20
1,150,8016 / PIL ANDINA S.A.,1314 / SAN RAFAEL--PIL ANDINA,L19,UCG 90-20


In [62]:
ids = list(tabla['id'])
ids

[151, 150]

In [63]:
def crear_lista_imagenes(id_proyecto, source_amigo_id, doc):
    print('buscando fotos')
    query_fotos = {'query': 'select * from gallery_67001 where source_amigo_id=\'{amigo_id}\''.format(amigo_id=source_amigo_id)}
    print(query_fotos)
    project = 'https://app.amigocloud.com/api/v1/projects/' + id_proyecto + '/sql'
    select_fotos = amigocloud.get(project, query_fotos)
    fotos = select_fotos['data']
    lista_fotos_inline = []
    print('descargando fotos')
    for foto in fotos:
        print('descargando una foto')
        response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + foto['s3_filename'])
        file = open(ruta + '/fotos/' + foto['s3_filename'], "wb")
        file.write(response.content)
        file.close()
        lista_fotos_inline.append({'foto': docxtpl.InlineImage(doc, image_descriptor=ruta+'/fotos/' + foto['s3_filename'], width=Mm(60))})
    return lista_fotos_inline

In [64]:
def generar_reporte(cod):
    id = cod
    query = {'query': 'select mu.*, (select count(*) from dataset_365891 da where mu.amigo_id = da.muestra_ref_datos) from dataset_365887 mu where id={idd}'.format(idd=id)}
    select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/35567/sql', query)
    data = select['data'][0]
    data['fecha_registro'] = datetime.strptime(data['fecha_registro'].split(' ')[0], '%Y-%m-%d').strftime('%d/%m/%Y')
    print(data)
    insp = {
        'fecha':data['fecha_registro'],
        'canhero':data['canhero'],
        'propiedad':data['propiedad'].split('--')[0],
        'lote':data['lote'],
        'area':data['area'],
        'variedad':data['variedad'],
        'edad':data['edad'],
        'ciclo':data['ciclo'],
        'num_mus':data['count']*20,
        'id':data['id'],
        'amigo_id':data['amigo_id']
    }
    insp = collections.namedtuple("insp", insp.keys())(*insp.values())  

    query = {'query': 'select * from dataset_365891 where muestra_ref_datos=\'{amigo_id}\''.format(amigo_id=data['amigo_id'])}
    select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/35567/sql', query)
    datos = select['data']
    tabla_datos = pd.DataFrame(datos)[['id', 'total_yemas', 'perforadas', 'enraizamiento', 'brotacion', 'muerta']]

    total_yemas = tabla_datos['total_yemas'].sum()
    total_perforadas = tabla_datos['perforadas'].sum()
    total_enraizamiento = tabla_datos['enraizamiento'].sum()
    total_brotacion = tabla_datos['brotacion'].sum()
    total_muertas = tabla_datos['muerta'].sum()
    total_inviables = total_perforadas + total_enraizamiento + total_brotacion + total_muertas
    total_viables = total_yemas - total_inviables

    porcen_perforadas = (total_perforadas / total_yemas)*100
    porcen_enraizamiento = (total_enraizamiento / total_yemas)*100
    porcen_brotacion = (total_brotacion / total_yemas)*100
    porcen_muertas = (total_muertas / total_yemas)*100
    porcen_inviables = (total_inviables / total_yemas)*100
    porcen_viables = (total_viables / total_yemas)*100

    res = {
        'diat':total_perforadas,
        'en':total_enraizamiento,
        'bro':total_brotacion,
        'mur':total_muertas,
        'invi':total_inviables,
        'via':total_viables,
        'ana':total_yemas,
        'diatp':porcen_perforadas,
        'enp':porcen_enraizamiento,
        'brop':porcen_brotacion,
        'murp':porcen_muertas,
        'invip':porcen_inviables,
        'viap':porcen_viables
    }
    res = collections.namedtuple("res", res.keys())(*res.values())
    
    doc = DocxTemplate(ruta + "/templates/tpl_rpt_calidad_semilla.docx")
    id_proyecto = '35567'
    source_amigo_id = insp.amigo_id
    inline_fotos = crear_lista_imagenes(id_proyecto, source_amigo_id, doc)
    context = {'insp':insp, 'res':res, 'fotos':inline_fotos }
    doc.render(context)
    ca = insp.canhero.split(' / ')
    file_name = ca[0] + '_CDSE_' + insp.fecha.replace('/', '-') + '_' + ca[1] + '_' + str(insp.id) + '_' + str(insp.lote)
    doc.save(ruta + '/informes/_' + file_name + '.docx')

In [65]:
ids

[151, 150]

In [66]:
for i in ids:
    generar_reporte(i)
    print(i)

{'fecha_registro': '25/02/2025', 'id': 151, 'estado': 'APROBADO', 'ubicacion': '0101000020E6100000D789CBF10A944FC0C4F7205F9D3231C0', 'tecnico_responsable': 'MILTON', 'canhero': '4171 / FRITZER CUELLAR SAMIR EDUARDO', 'lote': 'L7', 'observaciones': None, 'propiedad': '273 / HACIENDA LOS NIETOS--FRITZER', 'variedad': 'UCG 90-20', 'ciclo': 'HOJA', 'area': None, 'edad': None, 'usuario': '', 'amigo_id': 'da080ffdd6d74f8898e7c5eb14833e90', 'count': 1}
buscando fotos
{'query': "select * from gallery_67001 where source_amigo_id='da080ffdd6d74f8898e7c5eb14833e90'"}
descargando fotos
descargando una foto
151
{'fecha_registro': '25/02/2025', 'id': 150, 'estado': 'APROBADO', 'ubicacion': '0101000020E6100000598B4F01308B4FC01926AEBE157831C0', 'tecnico_responsable': 'MILTON', 'canhero': '8016 / PIL ANDINA S.A.', 'lote': 'L19', 'observaciones': None, 'propiedad': '1314 / SAN RAFAEL--PIL ANDINA', 'variedad': 'UCG 90-20', 'ciclo': 'HOJA', 'area': None, 'edad': 10, 'usuario': '', 'amigo_id': 'c0fc1dcb508

In [118]:
id = 29
query = {'query': 'select mu.*, (select count(*) from dataset_338313 da where mu.amigo_id = da.muestra_ref_datos) from dataset_338305 mu where id={idd}'.format(idd=id)}
select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/33953/sql', query)
data = select['data'][0]
data['fecha_registro'] = datetime.strptime(data['fecha_registro'].split(' ')[0], '%Y-%m-%d').strftime('%d/%m/%Y')    

In [119]:
data

{'fecha_registro': '08/03/2024',
 'id': 29,
 'ubicacion': '0101000020E6100000666CE8667FAF4FC0E919C231CB3A31C0',
 'canhero': '11860 / ROCA SERRANO ARMANDO',
 'lote': 'L3',
 'tipo': None,
 'observaciones': None,
 'amigo_id': '888354e0fd8441b0a27d8ef97386e1ec',
 'propiedad': '114 / SAUSALITO--ROCA',
 'variedad': 'UCG 90-20',
 'ciclo': 'HOJA',
 'area': 31.71,
 'edad': 10,
 'canha1': 0,
 'canha2': 0,
 'canha3': 0,
 'canha4': 0,
 'canha5': 0,
 'canha6': 0,
 'canha7': 0,
 'canha8': 0,
 'canha9': 0,
 'canha10': 0,
 'usuario': 'Bismark Socompi Rodríguez',
 'count': 10}

In [158]:
insp = {
    'fecha':data['fecha_registro'],
    'canhero':data['canhero'],
    'propiedad':data['propiedad'].split('--')[0],
    'lote':data['lote'],
    'area':data['area'],
    'variedad':data['variedad'],
    'edad':data['edad'],
    'ciclo':data['ciclo'],
    'num_mus':data['count']*20,
    'id':data['id']
}
insp = collections.namedtuple("insp", insp.keys())(*insp.values())
insp

insp(fecha='08/03/2024', canhero='11860 / ROCA SERRANO ARMANDO', propiedad='114 / SAUSALITO', lote='L3', area=31.71, variedad='UCG 90-20', edad=10, ciclo='HOJA', num_mus=200, id=29)

In [159]:
vi = {
    'uno': data['canha1'],
    'dos': data['canha2'],
    'tres': data['canha3'],
    'cuatro': data['canha4'],
    'cinco': data['canha5'],
    'seis': data['canha6'],
    'siete': data['canha7'],
    'ocho': data['canha8'],
    'nueve': data['canha9'],
    'diez': data['canha10']
}

vi_values = list(vi.values())
vi_sum = sum(vi_values)
vi['porcen'] = vi_sum
vi = collections.namedtuple("vi", vi.keys())(*vi.values())
vi

vi(uno=0, dos=0, tres=0, cuatro=0, cinco=0, seis=0, siete=0, ocho=0, nueve=0, diez=0, porcen=0)

In [160]:
query = {'query': 'select * from dataset_338313 where muestra_ref_datos=\'{amigo_id}\''.format(amigo_id=data['amigo_id'])}
select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/33953/sql', query)
datos = select['data']
tabla_datos = pd.DataFrame(datos)[['id', 'total_yemas', 'perforadas', 'enraizamiento', 'brotacion', 'muerta']]
tabla_datos

,id,total_yemas,perforadas,enraizamiento,brotacion,muerta
0,1763,331,10,2,2,0
1,1764,307,2,4,0,0
2,1765,300,6,0,5,0
3,1766,306,0,1,9,0
4,1767,274,0,2,7,0
5,1768,282,0,3,6,0
6,1769,291,0,1,6,0
7,1770,291,1,4,1,0
8,1771,296,0,13,0,0
9,1772,312,14,1,2,0


In [170]:
total_yemas = tabla_datos['total_yemas'].sum()
total_perforadas = tabla_datos['perforadas'].sum()
total_enraizamiento = tabla_datos['enraizamiento'].sum()
total_brotacion = tabla_datos['brotacion'].sum()
total_muertas = tabla_datos['muerta'].sum()
total_inviables = total_perforadas + total_enraizamiento + total_brotacion + total_muertas
total_viables = total_yemas - total_inviables

porcen_perforadas = (total_perforadas / total_yemas)*100
porcen_enraizamiento = (total_enraizamiento / total_yemas)*100
porcen_brotacion = (total_brotacion / total_yemas)*100
porcen_muertas = (total_muertas / total_yemas)*100
porcen_inviables = (total_inviables / total_yemas)*100
porcen_viables = (total_viables / total_yemas)*100

In [162]:
res = {
    'diat':total_perforadas,
    'en':total_enraizamiento,
    'bro':total_brotacion,
    'mur':total_muertas,
    'invi':total_inviables,
    'via':total_viables,
    'ana':total_yemas,
    'diatp':porcen_perforadas,
    'enp':porcen_enraizamiento,
    'brop':porcen_brotacion,
    'murp':porcen_muertas,
    'invip':porcen_inviables,
    'viap':porcen_viables
}
res = collections.namedtuple("res", res.keys())(*res.values())
res

res(diat=33, en=31, bro=38, mur=0, invi=102, via=2888, ana=2990, diatp=1.1036789297658862, enp=1.0367892976588629, brop=1.2709030100334449, murp=0.0, invip=3.411371237458194, viap=96.5886287625418)

In [166]:
doc = DocxTemplate(ruta + "/templates/tpl_rpt_calidad_semilla.docx")
context = {'insp':insp, 'vi':vi, 'res':res}
doc.render(context)
ca = insp.canhero.split(' / ')
file_name = ca[0] + '_CDS_' + insp.fecha.replace('/', '-') + '_' + ca[1] + '_' + str(insp.id)
doc.save(ruta + '/informes/' + file_name + '.docx')

In [165]:
insp

insp(fecha='08/03/2024', canhero='11860 / ROCA SERRANO ARMANDO', propiedad='114 / SAUSALITO', lote='L3', area=31.71, variedad='UCG 90-20', edad=10, ciclo='HOJA', num_mus=200, id=29)